## NOTE
This is notebook is merged code from multiple notebooks. I'm in the process of cleaning the rest of the code but this is the general flow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser
import seaborn as sns
from IPython.display import Image
import matplotlib
import os
import nltk
import csv
import sys
import matplotlib.dates as mdates


''' Settings '''
sns.set_style("white")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.precision', 3)
plt.style.use('fivethirtyeight')

In [ ]:
''' Load and concatinate CSV files into single dataframe'''
#csv.field_size_limit(sys.maxsize)

# Primary data
csv_names = ['/home/ubuntu/dev/project04-foia/data/foia_root.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path)
    )
df_root = pd.concat(files) 

# Comments
csv_names = ['/home/ubuntu/dev/project04-foia/data/foia_comm.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path)
    )
df_comm = pd.concat(files) 

# Files
csv_names = ['/home/ubuntu/dev/project04-foia/data/foia_file.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path)
    )
df_file = pd.concat(files) 

csv_names = ['/home/ubuntu/dev/project04-foia/data/foia_file_data_part.csv',"/home/ubuntu/dev/project04-foia/data/foia_file_data.csv",'/home/ubuntu/dev/project04-foia/data/foia_file_data_part2.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path)
    )
df_file_data = pd.concat(files) 

df_root = df_root.reset_index()
df_comm = df_comm.reset_index()
df_file = df_file.reset_index()
df_file_data = df_file_data.reset_index()

df = pd.merge(df_root, df_comm, on='id_key', how='left', suffixes=('_root', '_comm'))

Fix data

In [ ]:
csv_names = ['/home/ubuntu/dev/project04-foia/data/foia_agency.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path)
    )
df_gov = pd.concat(files)

# Get agecny for each row
def get_agency(row):
    return str(df_gov[df_gov['id']==row]['name'])
df_merge['agency_name'] = df_merge['agency'].apply(get_agency)
df = df_merge

# Dates
df['datetime_done'] = pd.to_datetime(df['datetime'])

Cleaning

In [ ]:
# Get topics for file data
df = df.dropna(subset=['subject'])

# Clean
df['subject'] = df['subject'].str.replace('\r', ' ')
df['subject'] = df['subject'].str.replace('\n', ' ')
df['subject'] = df['subject'].replace(r'[^a-zA-Z]',' ',regex=True)
df['subject'] = df['subject'].replace(r'[^a-zA-Z]',' ',regex=True)
df['subject'] = df['subject'].str.lower()

# Get topics for file data
df = df.dropna(subset=['communication'])

# Rough: remove names in validiction 
df['communication'] = df['communication'].str.rsplit(' ', 2).str[0]

# Clean

df['communication'] = df['communication'].str.replace('The requested documents will be made available to the general public and this request is not being made for commercial purposes', '')
df['communication'] = df['communication'].str.replace('In the event that there are fees I would be grateful if you would inform me of the total charges in advance of fulfilling my request', '')
df['communication'] = df['communication'].str.replace('I would prefer the request filled electronically by e-mail attachment if available or CD-ROM if not', '')
df['communication'] = df['communication'].str.replace('Thank you in advance for your anticipated cooperation in this matter', '')
df['communication'] = df['communication'].str.replace('I look forward to receiving your response to this request within 10 business days as the statute requires', '')


df['communication'] = df['communication'].str.replace('To Whom It May Concern', '')
df['communication'] = df['communication'].str.replace('Pursuant to the', '')
df['communication'] = df['communication'].str.replace('I hereby request the following records', '')
df['communication'] = df['communication'].str.replace('\r', ' ')
df['communication'] = df['communication'].str.replace('\n', ' ')
df['communication'] = df['communication'].replace(r'[^a-zA-Z]',' ',regex=True)
df['communication'] = df['communication'].replace(r'[^a-zA-Z]',' ',regex=True)
df['communication'] = df['communication'].str.lower()

# Get topics for file data
df = df_file_data
df = df.dropna(subset=['file'])
df['file'] = df['file'].str.replace('\r', ' ')
df['file'] = df['file'].str.replace('\n', ' ')
df['file'] = df['file'].replace(r'[^a-zA-Z]',' ',regex=True)
df['file'] = df['file'].replace(r'[^a-zA-Z]',' ',regex=True)
df['file'] = df['file'].str.lower()


Lemm

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

df['subject_lemm'] = df['subject'].apply(lemmatize_text)
df['communication_lemm'] = df['communication'].apply(lemmatize_text)
df['file_lemm'] = df['file'].apply(lemmatize_text)

Stop words

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
words = ['To Whom It May Concern',"nd","st","rd", "re", 'Pursuant to the','Information Act', 'I hereby request the following records','The requested documents will be made available to the general public and this request is not being made for commercial purposes. In the event that there are fees I would be grateful if you would inform me of the total charges in advance of fulfilling my request. I would prefer the request filled electronically  by e-mail attachment if available or CD-ROM if not. Thank you in advance for your anticipated cooperation in this matter. I look forward to receiving your response to this request within 10 business days  as the statute requires','sincerely','working', 'request', 'dear', 'mr', 'request', 'hello', 'request', 'send', 'http', 'status', 'link', 'sorry', 'mrs', 'sir', 'view', 'request',"Good morning", "Good Afternoon","Hello","hi","Attached please find",'The requested documents will be made available to the general public  and this request is not being made for commercial purposes','In the event that there are fees  I would be grateful if you would inform me of the total charges in advance of fulfilling my request. I would prefer the request filled electronically  by e-mail attachment if available or CD-ROM if not','Thank you in advance for your anticipated cooperation in this matter. I look forward to receiving your response to this request within 5 business days  as the statute requires']
words.extend("hi respond completion estimated time search information time search information provide office know need response documents let number requested thank contact reference cost report attached like data foiaonline gov foia https action public message regulations application confirm information submission follows objectid home concern muckrock requests com foiaonline password account user days reset inactivity log sending application message happening prevent lock using page whomever obviously excellent efforts attention statistical local grounds contain good morning afternoon check follow instructions page right clicking evening foia records information office greetings agency gov number attachments receipt contact does subject date government email thanks thank sent address just check forward yes received week records new message added open closed www".split(' '))
words.extend("send problem got individual request mailto inquiry general help link dear mr ms request view working processing currently possible appreciate soon patience queue case actively assured committed additional details request following processed final affirmed acknowledge grant completely muckrock com requests".split(' '))
words.extend('clarification needed clarification needed wanted copied originally originally submitted submitted copied originally expect ntended recipient use intended recipient confidential error privileged prohibited requester cmailto cmailto cmailto  advance available fees total event filled prefer fulfilling advance fulfilling completed previously'.split(' '))
words.extend('month later written format set works best considered appeal current  release likely expedited release terms people daily receive'.split(' '))
words = [w.lower() for w in words]

my_stop_words = ENGLISH_STOP_WORDS.union(words)

Bag of words

In [ ]:
params = {"unit":"suject_lemm",
          "countv":{
            "min_df":.000025,
            "max_df":0.1,
            "ngram_range":(1,3)
            },
          "tfidf":{
            "min_df":.000035,
            "max_df":0.1,
            "ngram_range":(1,3)
            },
          "lda":{
              "n_components" : 25,
              "max_iter": 30
          },
          "nmf":{
              "n_components" : 25,
              "max_iter": 200
          }}

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

count_vectorizer = CountVectorizer(
    stop_words=my_stop_words,
    min_df=params['countv']['min_df'],
    max_df=params['countv']['max_df'],
    token_pattern="\\b[a-z][a-z]+\\b",
    ngram_range=params['countv']['ngram_range']
)
count_vectorizer_fit = count_vectorizer.fit_transform(df[params['unit']])

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words,
                                   min_df=params['tfidf']['min_df'],
                                   max_df=params['tfidf']['max_df'],
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   ngram_range=params['tfidf']['ngram_range'])
tfidf_vectorizer_fit = tfidf_vectorizer.fit_transform(df[params['unit']])

Weights

In [ ]:
from sklearn import decomposition

lda = decomposition.LatentDirichletAllocation(
    n_components=params['lda']['n_components'], 
    learning_method="batch", 
    verbose=1, 
    max_iter=params['lda']['max_iter'],
    n_jobs=-1
)

lda.fit(count_vectorizer_fit)
lda_model = lda.transform(count_vectorizer_fit)

In [ ]:
import numpy as np
from sklearn.decomposition import NMF

nmf = NMF(n_components=params['nmf']['n_components'], 
          init='random', 
          max_iter=params['nmf']['max_iter'])
nmf_fit = nmf.fit_transform(tfidf_vectorizer_fit)

Topics

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

def get_top_words_csv(model, feature_names, n_top_words,model_type,weight_type,col):
    topics={}
    for topic_idx, topic in enumerate(model.components_):
        message = ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        printmd(f"**Topic #{topic_idx}**: {message}")
        topics[topic_idx] = message
    df_temp = pd.DataFrame(list(topics.items()),columns=['topic_id', 'content'])
    df_temp['type']         = col
    df_temp['model']        = model_type
    df_temp['n_components'] = params[model_type]['n_components']
    df_temp['max_iter']     = params[model_type]['max_iter']
    df_temp['weight']       = weight_type
    df_temp['min_df']       = params[weight_type]['min_df']
    df_temp['ngram_range']  = str(params[weight_type]['ngram_range'])
    df_results.append(df_temp)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        printmd(f"**Topic #{topic_idx}**: {message}")
    print()


In [ ]:
# Visualize with pyLDAvis to help produce topics
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, count_vectorizer_fit, count_vectorizer)

In [ ]:
# Save current topics to csv

get_top_words_csv(lda, list(count_vectorizer.get_feature_names()), 20,'lda','countv','subject')
get_top_words_csv(nmf, list(tfidf_vectorizer.vocabulary_.keys()), 20,'nmf','tfidf','subject')

# Generated topics
topics = {
0:"Trump Imigration Policies",
1: "Jenny Burke",
2: "Palantir",
3: "Massachusettes Police Protests",
4: "Muslim Ban",
5: "?",
6: "Magnolia Patrol",
7: "Police Drones",
8: "Nesbitt Research",
9: "Benedict Anderson",    
10: "March On Washington",
11: "Aircraft Attacks",
12: "Defense Director",
13: "?",
14: "Homeland Security",
15: "FBI Records",
16: "Solitary Confinemnt",
17: "Internet Surviellence",
18: "Trump Transition",
19: "FDA Food Ingredients",
20: "Body Cam Footage",
21: "Whistleblower",
22: "Predictive Policing",
23: "Transportation Security",
24: "FCC & ICE"
}


In [ ]:
# Print topic weights
def plot_topic_matrix(model,vectorizer_feature):

   disp_topics = pd.DataFrame()
   prob_comp = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]
   for topic_idx, topic in enumerate(prob_comp):
       top_indices = topic.argsort()[:-30 - 1:-1]
       top_words = [f"{vectorizer_feature[i]} {topic[i]:.1%}" for i in top_indices]
       disp_topics[f"Topic {topic_idx}"] = top_words
   return disp_topics
plot_topic_matrix(lda,count_vectorizer.get_feature_names())

In [ ]:
# Plot topic weights
top_n_words = 10
t_words, word_strengths = {}, {}
for t_id, t in enumerate(nmf.components_):
    t_words[t_id] = [tfidf_vectorizer.get_feature_names()[i] for i in t.argsort()[:-top_n_words - 1:-1]]
    word_strengths[t_id] = t[t.argsort()[:-top_n_words - 1:-1]]
t_words

fig, ax = plt.subplots(figsize=(10,18), ncols=2, nrows=5)
plt.subplots_adjust(
    wspace  =  0.5,
    hspace  =  0.5
)
c=0
for row in range(0,5):
    for col in range(0,2):
        sns.barplot(x=word_strengths[c], y=t_words[c], color="red", ax=ax[row][col])
        c+=1
plt.show()

Explore

In [ ]:
# TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(lda_model[::40])

In [ ]:
# plot tsne!
doc_max_top = np.argmax(lda_model, axis=1 ) 
doc_max_top.shape
plt.figure(figsize=(12,12))
plt.scatter(X_tsne[:,0],X_tsne[:,1], c=doc_max_top, alpha=1,s=2)
plt.title("multicore-tsne + nmf + subject with max topic labels");
plt.legend(np.arange(1,10));
#plt.savefig("tsne w euc.pdf")

Clustering

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering

clustering = SpectralClustering(n_clusters=10)
clusters = clustering.fit_predict(lda_model[::100])
pca = PCA(2) 
projected = pca.fit_transform(lda_model[::100])

plt.scatter(projected[:, 0], projected[:, 1],
            c=list(clusters), edgecolor='none', alpha=0.5)
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();


data = pd.DataFrame()
data['x']=projected.T[0]
data['y']=projected.T[1]
data['labels']= list(clusters)

facet = sns.lmplot(data=data, x='x', y='y', hue='labels', 
                   fit_reg=False, legend=True, legend_out=True, size=12, aspect=1)

Charts

In [ ]:
plt.style.use('fivethirtyeight')

df = pd.read_csv('/home/ubuntu/dev/project04-foia/data/USE_THIS_DATAFRAME.csv')
df = df.sort_values(by=['price'],ascending=False)
df = df[df['price']!= 0]

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.sort_values(by=['price'],ascending=False)

# group by price
a = df.groupby(['topic'])['price'].median()
a = pd.DataFrame(a).reset_index()
a.columns = ["topic","price"]
a = a.sort_values(by="price",ascending=True)
a = a[10:]
a = a[a["topic"] != "?"]


fig = plt.figure(figsize=(12,8),dpi=500,facecolor='w', edgecolor='k')
ax = fig.add_subplot(1, 1, 1)

plt.barh(a['topic'],a['price'],color='k')
plt.xlabel("Media price per FOIA")
plt.ylabel("Topic")

csfont = {}
hfont = {}

plt.xticks(**hfont,fontsize=14)
plt.yticks(**hfont,fontsize=14)
ax.set_xscale('log')
for i, v in enumerate(a['price']):
    ax.text(v+5 , i,' ${:,.2f}'.format(v),va="center", color='blue', fontweight='bold')
plt.xlim(10**0,10**6)
plt.show()


